# Notes By Frequency In Corpus

## Goal
Which scale degrees are most common in the corpus? Output a collection of scale degrees by frequency of occurence.

## Idea
1. Access each piece in the corpus.
    * create a list of elements in the corpus
2. For each piece
    * convert the piece to a standard key based on mode
    * break piece into notes
    * for each note, record an instance of represented scale degree.
3. Output the recorded scale degree collection

## Necessary Libraries

In [1]:
import music21 as mus

## Helper Functions


### Entire Corpus

#### listElementsOfCorpus(corpus_chosen):
        returns list of pieces in corpus

In [4]:
def listElementsOfCorpus(corpus_chosen):
    element_list = []
    corpus_accessed = mus.corpus.getComposer(corpus_chosen)
    length_corpus_list = len(corpus_accessed)
    i = 0
    while i < length_corpus_list:
        list_of_pieces = mus.corpus.parse(corpus_accessed[i])
        j = len(list_of_pieces)
        for k in range(0,j):
            element_list.append(list_of_pieces[k])
        i+=1
    return(element_list)

### Single Piece

#### transposeMelody( melody_line, melody_key, final_major_key, final_minor_key)
* Input: Flattened melody w/out repeats
* Output: Melody transposed to correct major or minor output

In [27]:
def transposeMelody( melody_line , melody_key, final_major_key , final_minor_key ):
    if melody_key.mode == 'major':
        return(subTransposeMelody( melody_line , melody_key.tonic , final_major_key))
    elif  melody_key.mode == 'minor':
        return(subTransposeMelody( melody_line , melody_key.tonic , final_minor_key))                 
    else:
        print('Failed to tranpose melody.\n')

#### subTransposeMelody( melody_line, melody_tonic, final_key)
* given melody in major key, transpose it to the final key

In [28]:
def subTransposeMelody( melody_line, melody_tonic , final_key ):
    current_tonic = mus.note.Note(melody_tonic)
    final_tonic = mus.note.Note(final_key)
   
          
    interval_wanted = mus.interval.notesToChromatic( current_tonic , final_tonic )
    transposed_melody = melody_line.flat.transpose(interval_wanted.getDiatonic().directedName)
    
    return(transposed_melody)

#### countDegrees( pitchList , minor_dict , major_dict , mode)
    * Input: Flattened pitch list, dictionaries
    * Output: countDegreeOccurrence for mode

In [89]:
def countDegrees( pitch_list , minor_dict, major_dict, mode, maj_scale , min_scale):
    if mode == 'major':
        major_dict = countDegOccurrence( pitch_list , major_dict , maj_scale)
    elif mode == "minor":
        minor_dict = countDegOccurrence(  pitch_list , minor_dict, min_scale)
    else:
        print('Neither major or minor key. Error counting degrees.')
    return([major_dict, minor_dict])

#### countDegOccurrence( melody_line , dictionary )
given a flattened melody line and a dictionary, count the occurences of scale degrees scanned


In [100]:
def countDegOccurrence( pitch_list, dictionary , final_scale):
    for p in pitch_list:
        degP = final_scale.getScaleDegreeFromPitch(p)
        if degP in dictionary.keys():
            dictionary[degP] += 1
        else:
            dictionary[degP] = 1
    return(dictionary)
    

## Testing

### General

#### Constants

In [107]:
corpus_chosen = 'airdsAirs'
final_major_key = 'C5'#'F#4' 
final_minor_key = 'A4'#'Eb4'

### Universal Initializations

In [119]:
majScale = mus.scale.MajorScale(final_major_key)
minScale = mus.scale.MinorScale(final_minor_key)

minorDict = {}
majorDict = {}

### One piece

#### List of pieces in corpus

In [7]:
pieceList = listElementsOfCorpus(corpus_chosen)

#### Single Piece, Melody Line

In [66]:
piece0 = pieceList[0]
piece1 = pieceList[3]
#piece0.show('lily')
#piece1.show('lily')

<music21.stream.Stream 0x14f0329a6a0>

#### Piece formatting (no repeats, only one melody line)

In [39]:
piece0 = piece0.expandRepeats()
piece0 = piece0.flat
#piece0.show('lily')

piece1 = piece1.expandRepeats()
peice1 = piece1.flat
piece1.show('lily')

#### Analyze the piece's key

In [40]:
p0key = piece0.analyze('key')
p1key = piece1.analyze('key')
p1key

<music21.key.Key of e minor>

#### Transpose to common key

In [43]:
p0Transpose = transposeMelody(piece0, p0key, final_major_key, final_minor_key)
p0Transpose.show('lily')

p1Transpose = transposeMelody(piece1, p1key, final_major_key, final_minor_key)
p1Transpose.show('lily')
p1Transpose.show('midi')

#### Count degree occurrence 

In [106]:
pitchList0 = p0Transpose.pitches
shortPitchList0 = [str(p) for p in pitchList0]

pitchList1 = p1Transpose.pitches
shortPitchList1 = [str(p) for p in pitchList1]

#print(type(p0key.tonic.name))
#scale0 = mus.scale.MajorScale(p0key.tonic.name)
#print(scale0)

# WRONG SCALE! NEED FINAL SCALE
majScale = mus.scale.MajorScale(final_major_key)
minScale = mus.scale.MinorScale(final_minor_key)

minorDict = {}
majorDict = {}

degOccurrence = countDegrees(shortPitchList0, minorDict, majorDict, p0key.mode , majScale, minScale)
degOccurrence

[{1: 40, 2: 34, 3: 52, 4: 8, 5: 14, 6: 6, 7: 14}, {}]

In [105]:
degOccurrence2 = countDegrees(shortPitchList1, minorDict, majorDict, p1key.mode , majScale, minScale)
degOccurrence2

[{1: 40, 2: 34, 3: 52, 4: 8, 5: 14, 6: 6, 7: 14},
 {1: 70, 2: 32, 3: 70, 4: 26, 5: 48, 6: 6, 7: 12}]

### Set of pieces/ corpus

In [122]:
for pieceNo in range(0,len(pieceList)):
    
    melody = pieceList[pieceNo]#.expandRepeats()
    #melody = melody.flat
    
    melodyKey = melody.analyze('key')
    
    melodyTranspose = transposeMelody(melody, melodyKey, final_major_key, final_minor_key)
    
    pitchList = melodyTranspose.pitches
    shortPitchList = [str(p) for p in pitchList]
    
    [majorDict, minorDict] = countDegrees(shortPitchList, minorDict, majorDict, melodyKey.mode, majScale, minScale)
    

In [123]:
majorDict

{1: 21849, 2: 14143, 3: 18362, 4: 9635, 5: 17099, 6: 8187, 7: 5608, None: 675}

In [124]:
minorDict

{1: 4816, 2: 2651, 3: 5226, 4: 4180, 5: 4364, 6: 732, 7: 3093, None: 525}

# To Do
* Clean up code
* Make this a function with clear input and ooutput
* Make the dictionaries importable to other files
* Figure out how these frequencies of occurence correspond to probabilities
* Figure out what I want to do with rests/no pitch
* Understand why expandRepeats and flattened melody don't work
* Use the dictionary as input into something else!!! 
* Figure out how bins work
* Can we do anythin